In [22]:
import data_process as dp
import data_graph as dg

# establish universal regions, plus a scrap bin
region_list = ["w", "m", "s", "n", "empty"]

# request dataset from online and store it in a variable
# data_fetch.write_to_csv(
# "https://www.ncei.noaa.gov/archive/archive-management-system/OAS/bin/prd/jquery/download/209268.17.17.tar.gz",
# "209268.17.17.tar.gz",
# )

disaster_data = dp.read_csv_to_var(
    "./0209268/17.17/data/0-data/events-US-1980-2023.csv"
)

# modify dates to be less specific years
dp.parse_all_years(disaster_data)

# convert the raw data (sorted by region) into graphable blocks of data
region_dict = dp.fill_all_regions(disaster_data, region_list)
cost_of_regions, deaths_of_regions = dp.organize_regions(region_dict)


AttributeError: module 'data_process' has no attribute 'organize_regions'

now we test print!

In [16]:
print(cost_of_regions["s"])

{'1980': [['Flooding', 2678.6, 7.0], ['Tropical Cyclone', 2177.2, 13.0]], '1981': [['Freeze', 2019.2, 0.0]], '1982': [['Flooding', 4807.9, 45.0]], '1983': [['Tropical Cyclone', 9180.0, 21.0], ['Drought', 9307.3, 0.0]], '1985': [['Freeze', 3492.0, 0.0], ['Tropical Cyclone', 3684.6, 4.0]], '1986': [['Drought', 5050.2, 100.0]], '1989': [['Severe Storm', 1367.6, 21.0], ['Tropical Cyclone', 23584.8, 97.0], ['Freeze', 5080.0, 10.0]], '1990': [['Flooding', 2396.3, 13.0]], '1992': [['Tropical Cyclone', 58860.0, 61.0], ['Severe Storm', 1418.9, 26.0]], '1993': [['Drought', 2704.8, 16.0]], '1994': [['Winter Storm', 6264.7, 9.0], ['Tropical Cyclone', 2066.9, 32.0], ['Flooding', 2050.0, 19.0]], '1995': [['Severe Storm', 1161.5, 0.0], ['Tropical Cyclone', 11086.5, 33.0]], '1996': [['Tropical Cyclone', 9763.8, 37.0]], '1997': [['Severe Storm', 1877.0, 67.0]], '1998': [['Tropical Cyclone', 14409.3, 21.0], ['Drought', 6703.3, 200.0], ['Flooding', 1761.7, 31.0]], '1999': [['Tropical Cyclone', 11878.5, 7